# **Notebook Data Wrangling**
---

Untuk kebutuhan Thread Twitter Pacmann

In [1]:
import pandas as pd

In [24]:
# read data
rental_data = pd.read_csv("rental_data.csv")

rental_data.head()

,Unnamed: 0,rental_id,customer_id,rental_date,rental_status,return_date
0,0,fEep-28587-734.,743,2017-04-05,Returned,2017-04-14
1,1,uNLr-14032-611.,806,2017-07-13,Rented,NaN
2,2,PfzU-19322-932.,230,2015-04-25,Returned,2015-05-01
3,3,cOLR-80286-356.,369,2023-02-06,Rented,NaN
4,4,ZbEy-33182-842.,363,2016-07-06,Returned,2016-07-17


Check missing values

In [25]:
rental_data.isna().sum()

Unnamed: 0         0
rental_id          0
customer_id        0
rental_date        0
rental_status      0
return_date      515
dtype: int64

Drop kolom `Unnamed: 0` karena tidak dibutuhkan

In [26]:
rental_data = rental_data.drop(["Unnamed: 0"], axis = 1)

rental_data.head()

,rental_id,customer_id,rental_date,rental_status,return_date
0,fEep-28587-734.,743,2017-04-05,Returned,2017-04-14
1,uNLr-14032-611.,806,2017-07-13,Rented,NaN
2,PfzU-19322-932.,230,2015-04-25,Returned,2015-05-01
3,cOLR-80286-356.,369,2023-02-06,Rented,NaN
4,ZbEy-33182-842.,363,2016-07-06,Returned,2016-07-17


### **Remove dot (.) from `rental_id`**
---

Karena `rental_id` memiliki tipe data `object`, maka kita perlu convert menjadi `str` terlebih dahulu baru bisa menghilangkan `dot(.)`

In [27]:
rental_data["rental_id"] = rental_data["rental_id"].str.replace(".", "")

rental_data.head()

,rental_id,customer_id,rental_date,rental_status,return_date
0,fEep-28587-734,743,2017-04-05,Returned,2017-04-14
1,uNLr-14032-611,806,2017-07-13,Rented,NaN
2,PfzU-19322-932,230,2015-04-25,Returned,2015-05-01
3,cOLR-80286-356,369,2023-02-06,Rented,NaN
4,ZbEy-33182-842,363,2016-07-06,Returned,2016-07-17


### **Change Value in `rental_status`**
---

Cek sebelum mapping values

In [28]:
rental_data["rental_status"].value_counts().to_dict()

{'Rented': 486, 'Returned': 387, 'Late': 61, 'Overdue': 37, 'Returns': 29}

Membuat function yang digunakan untuk proses Mapping Values

In [29]:
def mapping_values(data):
    rental_data = data
    
    REPLACE_VALUES = {
        "Overdue": "Late",
        "Returns": "Returned"
    }
    
    rental_data = rental_data.replace(REPLACE_VALUES)
    
    return rental_data

In [30]:
rental_data = rental_data.apply(mapping_values)

Cek setelah mapping values

In [31]:
rental_data["rental_status"].value_counts().to_dict()

{'Rented': 486, 'Returned': 416, 'Late': 98}

### **Extract Year and Month in `rental_date`**
---

Convert it to `datetime` first

In [32]:
rental_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rental_id      1000 non-null   object
 1   customer_id    1000 non-null   int64 
 2   rental_date    1000 non-null   object
 3   rental_status  1000 non-null   object
 4   return_date    485 non-null    object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [34]:
rental_data["rental_date"] = pd.to_datetime(rental_data["rental_date"])

rental_data["rental_date"]

0     2017-04-05
1     2017-07-13
2     2015-04-25
3     2023-02-06
4     2016-07-06
         ...    
995   2020-07-03
996   2020-12-01
997   2020-01-30
998   2020-06-01
999   2015-02-25
Name: rental_date, Length: 1000, dtype: datetime64[ns]

In [35]:
rental_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rental_id      1000 non-null   object        
 1   customer_id    1000 non-null   int64         
 2   rental_date    1000 non-null   datetime64[ns]
 3   rental_status  1000 non-null   object        
 4   return_date    485 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 39.2+ KB


Extract `Year` dan `Month` dengan menggunakan method `dt`

In [36]:
rental_data["extract_rental_year"] = rental_data["rental_date"].dt.year
rental_data["extract_rental_month"] = rental_data["rental_date"].dt.month

In [37]:
rental_data.head()

,rental_id,customer_id,rental_date,rental_status,return_date,extract_rental_year,extract_rental_month
0,fEep-28587-734,743,2017-04-05,Returned,2017-04-14,2017,4
1,uNLr-14032-611,806,2017-07-13,Rented,NaN,2017,7
2,PfzU-19322-932,230,2015-04-25,Returned,2015-05-01,2015,4
3,cOLR-80286-356,369,2023-02-06,Rented,NaN,2023,2
4,ZbEy-33182-842,363,2016-07-06,Returned,2016-07-17,2016,7


### **Save it to different table**
---

- Filter data terlebih dahulu berdasarkan `rental_status`:
    - `Returned`
    - `Rented`
    - `Late`

In [38]:
rental_returned_data = rental_data[rental_data["rental_status"] == "Returned"]

rental_returned_data.head()

,rental_id,customer_id,rental_date,rental_status,return_date,extract_rental_year,extract_rental_month
0,fEep-28587-734,743,2017-04-05,Returned,2017-04-14,2017,4
2,PfzU-19322-932,230,2015-04-25,Returned,2015-05-01,2015,4
4,ZbEy-33182-842,363,2016-07-06,Returned,2016-07-17,2016,7
6,bRbe-62231-802,53,2017-08-02,Returned,2017-08-06,2017,8
7,KAuP-78858-794,886,2022-12-14,Returned,2022-12-25,2022,12


In [39]:
rental_rented_data = rental_data[rental_data["rental_status"] == "Rented"]

rental_rented_data.head()

,rental_id,customer_id,rental_date,rental_status,return_date,extract_rental_year,extract_rental_month
1,uNLr-14032-611,806,2017-07-13,Rented,NaN,2017,7
3,cOLR-80286-356,369,2023-02-06,Rented,NaN,2023,2
5,rUNA-37125-575,8,2020-10-31,Rented,NaN,2020,10
8,pwEX-08327-691,975,2023-01-27,Rented,NaN,2023,1
12,IAvq-81643-870,169,2023-03-23,Rented,NaN,2023,3


In [40]:
rental_late_data = rental_data[rental_data["rental_status"] == "Late"]

rental_late_data.head()

,rental_id,customer_id,rental_date,rental_status,return_date,extract_rental_year,extract_rental_month
22,RPNn-48420-671,829,2015-04-24,Late,2015-04-25,2015,4
24,Hzhs-50547-102,163,2020-09-22,Late,2020-10-04,2020,9
44,NtJF-60489-705,869,2015-06-16,Late,2015-07-10,2015,6
69,RUvf-62160-670,710,2016-06-25,Late,2016-07-19,2016,6
70,gake-10221-119,516,2016-07-17,Late,2016-08-15,2016,7


### **Load Data**
---

Simpan ke dalam bentuk csv

In [41]:
rental_returned_data.to_csv("rental_returned_data.csv", index = False)
rental_rented_data.to_csv("rental_rented_data.csv", index = False)
rental_late_data.to_csv("rental_late_data.csv", index = False)